In [ ]:
class VESC:
    ''' 
    VESC Motor controler using pyvesc
    This is used for most electric scateboards.
    
    inputs: serial_port---- port used communicate with vesc. for linux should be something like /dev/ttyACM1
    has_sensor=False------- default value from pyvesc
    start_heartbeat=True----default value from pyvesc (I believe this sets up a heartbeat and kills speed if lost)
    baudrate=115200--------- baudrate used for communication with VESC
    timeout=0.05-------------time it will try before giving up on establishing connection
    
    percent=.2--------------max percentage of the dutycycle that the motor will be set to
    outputs: none
    
    uses the pyvesc library to open communication with the VESC and sets the servo to the angle (0-1) and the duty_cycle(speed of the car) to the throttle (mapped so that percentage will be max/min speed)
    
    Note that this depends on pyvesc, but using pip install pyvesc will create a pyvesc file that
    can only set the speed, but not set the servo angle. 
    
    Instead please use:
    pip install git+https://github.com/LiamBindle/PyVESC.git@master
    to install the pyvesc library
    '''
    def __init__(self, serial_port, percent=.2, has_sensor=False, start_heartbeat=True, baudrate=115200, timeout=0.05, steering_scale = 1.0, steering_offset = 0.0 ):
        
        try:
            import pyvesc
        except Exception as err:
            print("\n\n\n\n", err, "\n")
            print("please use the following command to import pyvesc so that you can also set")
            print("the servo position:")
            print("pip install git+https://github.com/LiamBindle/PyVESC.git@master")
            print("\n\n\n")
            time.sleep(1)
            raise
        
        assert percent <= 1 and percent >= -1,'\n\nOnly percentages are allowed for MAX_VESC_SPEED (we recommend a value of about .2) (negative values flip direction of motor)'
        self.steering_scale = steering_scale
        self.steering_offset = steering_offset
        self.percent = percent
        
        try:
            self.v = pyvesc.VESC(serial_port, has_sensor, start_heartbeat, baudrate, timeout)
        except Exception as err:
            print("\n\n\n\n", err)
            print("\n\nto fix permission denied errors, try running the following command:")
            print("sudo chmod a+rw {}".format(serial_port), "\n\n\n\n")
            time.sleep(1)
            raise
        
    def run(self, angle, throttle):
        self.v.set_servo((angle * self.steering_scale) + self.steering_offset)
        self.v.set_duty_cycle(throttle*self.percent)

In [ ]:
v=VESC("/dev/tty.usbmodem3041")

In [ ]:
v.run(0,0)

In [ ]:
def make_points(image, average):
    print(average)
    slope, y_int = average
    y1 = image.shape[0]
    #how long we want our lines to be --> 3/5 the size of the image
    y2 = int(y1 * (3/5))
    #determine algebraically
    x1 = int((y1 - y_int) // slope)
    x2 = int((y2 - y_int) // slope)
    return np.array([x1, y1, x2, y2])
def average(image, lines):
    left = []
    right = []
    for line in lines:
#         print(line)
        x1, y1, x2, y2 = line.reshape(4)
        #fit line to points, return slope and y-int
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        print(parameters)
        slope = parameters[0]
        y_int = parameters[1]
        #lines on the right have positive slope, and lines on the left have neg slope
        if slope < 0:
            left.append((slope, y_int))
        else:
            right.append((slope, y_int))
    #takes average among all the columns (column0: slope, column1: y_int)
    right_avg = np.average(right, axis=0)
    left_avg = np.average(left, axis=0)
    #create lines based on averages calculates
    left_line = make_points(image, left_avg)
    right_line = make_points(image, right_avg)
    return np.array([left_line, right_line])
def display_lines(image, lines):
    lines_image = np.zeros_like(image)
    #make sure array isn't empty
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line
            #draw lines on a black image
            cv2.line(lines_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return lines_image

def roi(image):
    height = image.shape[0]
    polygons = np.array([
    [(100, height), (1000, height), (450, 150)]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    return mask

In [ ]:

# Apply Canny Filter to ROI
import cv2
import numpy as np

state = 0
maxState = 3

font = cv2.FONT_HERSHEY_SIMPLEX
  
# org
org = (50, 50)
# fontScale
fontScale = 1
# Blue color in BGR
color = (255, 0, 0)
# Line thickness of 2 px
thickness = 2


from matplotlib import pyplot as plt

def nothing(x):
    pass

iw = "Image Window"
tw = "Settings Window"
cv2.namedWindow(tw)
windowWidth = 1000
cv2.imshow(tw, np.ones([1,windowWidth]))
cv2.namedWindow(iw)

cv2.createTrackbar('low',tw,0,255,nothing)
cv2.createTrackbar('high',tw,0,255,nothing)
cv2.createTrackbar('kernel',tw,2,10,nothing)
cv2.setTrackbarMin('kernel', tw, 2)
cv2.createTrackbar('erosion',tw,0,10,nothing)
cv2.createTrackbar('dilation',tw,0,10,nothing)

cam_capture = cv2.VideoCapture(0)
origShape = cam_capture.read()[1].shape
#r = (0,0,origShape[1], origShape[1])
_, original = cam_capture.read()
r = cv2.selectROI(iw, original)

while True:
    _, original = cam_capture.read()
    
    #Rectangle marker
    rect_img = original[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])]
    sketcher_rect = rect_img
    
    kernel = cv2.getTrackbarPos('kernel',tw)
#     print(kernel)
    img = cv2.blur(rect_img,(kernel,kernel))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    erosion = cv2.getTrackbarPos('erosion',tw)
    dilation = cv2.getTrackbarPos('dilation',tw)
    
    
    x = cv2.getTrackbarPos('low',tw)
    y = cv2.getTrackbarPos('high',tw)
    edge = cv2.Canny(gray, x, y)
    edge = cv2.erode(edge,np.ones((kernel,kernel),np.uint8),iterations= erosion)
    edge = cv2.dilate(edge,np.ones((kernel,kernel),np.uint8),iterations= dilation)
    edge_roi = roi(edge)
#     txt = str(x,y,kernel)
    
    sketcher_rect = edge #sketch(sketcher_rect)
    #Conversion for 3 channels to put back on original image (streaming)
    sketcher_rect_rgb = cv2.cvtColor(sketcher_rect, cv2.COLOR_GRAY2RGB)

    #Replacing the sketched image on Region of Interest
    original[int(r[1]):int(r[1]+r[3]), int(r[0]):int(r[0]+r[2])] = sketcher_rect_rgb
    #
    
    imout = original
    if state==0:
        imout = original
    elif state == 1:
        imout = gray
    elif state == 2:
        imout = edge
    elif state == 3:
        #==============================================================================
        lines = cv2.HoughLinesP(edge, 2, np.pi/180, 50,np.array([]),minLineLength=20, maxLineGap=15)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 5)
            #cv2.imshow(iw, img)# blur
        #==============================================================================
        try:
            averageLines = average(img, lines)
        except:
            pass
        greenLines = display_lines(img,averageLines)
        
        imout = cv2.add(img, greenLines)
#     elif state == 4:
#         pass

    txt = "state:{} low:{} high:{} kernel:{} dilation:{} erosion:{}".format(state,x,y,kernel,dilation,erosion)
#     imout = cv2.drawLines(lines, imout)
    imout = cv2.putText(imout, txt, org, font, fontScale, color, thickness, cv2.LINE_AA)
    cv2.imshow(iw, imout)
    
    k = cv2.waitKey(1)
    if k == 113:
        break
    elif k == 116:
        print('t')
        _, frame = cam_capture.read()
        cv2.imshow(iw, frame)
        r = cv2.selectROI(iw,frame)    
    elif k == -1:
        pass
    elif k == 2 or k == 97:
        #left key or 'A' key
        state -=1
    elif k == 3 or k == 100:
        #right key or 'D' key
        print("right")
        state +=1 
    else:
        print(k)
        
    if state>maxState:
        state = 0
    if state<0:
        state= maxState
#     print(r)

cv2.destroyAllWindows()


In [ ]:
path = 'solidYellowCurve2.jpeg'
im = cv2.imread(path)
plt.imshow(im)

In [ ]:
# This is testing lines for live video
# https://pysource.com/2018/03/07/lines-detection-with-hough-transform-opencv-3-4-with-python-3-tutorial-21/

import cv2
import numpy as np
cam_capture = cv2.VideoCapture(0)
while cam_capture.isOpened():
    print("looping")
    ret, orig_frame = cam_capture.read()
    if not ret:
        cam_capture = cv2.VideoCapture(0)
        continue
    frame = cv2.GaussianBlur(orig_frame, (5, 5), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    low_yellow = np.array([18, 94, 140])
    up_yellow = np.array([48, 255, 255])
    mask = cv2.inRange(hsv, low_yellow, up_yellow)
    edges = cv2.Canny(mask, 75, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, maxLineGap=50)
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
        cv2.imshow("frame", frame)# blur
    cv2.imshow("edges", edges)
    key = cv2.waitKey(1)
    if key == 27:
        break
cv2.destroyAllWindows()

In [ ]:
def grey(image):
  #convert to grayscale
    image = np.asarray(image)
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

  #Apply Gaussian Blur --> Reduce noise and smoothen image
def gauss(image):
    return cv2.GaussianBlur(image, (5, 5), 0)

  #outline the strongest gradients in the image --> this is where lines in the image are
def canny(image):
    edges = cv2.Canny(image,50,150)
    return edges

In [ ]:
def region(image):
    height, width = image.shape
    #isolate the gradients that correspond to the lane lines
    triangle = np.array([
                       [(100, height), (475, 325), (width, height)]
                       ])
    #create a black image with the same dimensions as original image
    mask = np.zeros_like(image)
    #create a mask (triangle that isolates the region of interest in our image)
    mask = cv2.fillPoly(mask, triangle, 255)
    mask = cv2.bitwise_and(image, mask)
    return mask

In [ ]:
def display_lines(image, lines):
    lines_image = np.zeros_like(image)
    #make sure array isn't empty
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line
            #draw lines on a black image
            cv2.line(lines_image, (x1, y1), (x2, y2), (255, 0, 0), 10)
    return lines_image

In [ ]:
def average(image, lines):
    left = []
    right = []
    for line in lines:
        print(line)
        x1, y1, x2, y2 = line.reshape(4)
        #fit line to points, return slope and y-int
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        print(parameters)
        slope = parameters[0]
        y_int = parameters[1]
        #lines on the right have positive slope, and lines on the left have neg slope
        if slope < 0:
            left.append((slope, y_int))
        else:
            right.append((slope, y_int))
    #takes average among all the columns (column0: slope, column1: y_int)
    right_avg = np.average(right, axis=0)
    left_avg = np.average(left, axis=0)
    #create lines based on averages calculates
    left_line = make_points(image, left_avg)
    right_line = make_points(image, right_avg)
    return np.array([left_line, right_line])
    

In [ ]:
def make_points(image, average):
    print(average)
    slope, y_int = average
    y1 = image.shape[0]
    #how long we want our lines to be --> 3/5 the size of the image
    y2 = int(y1 * (3/5))
    #determine algebraically
    x1 = int((y1 - y_int) // slope)
    x2 = int((y2 - y_int) // slope)
    return np.array([x1, y1, x2, y2])

In [ ]:
copy = np.copy(im)
edges = cv2.Canny(copy,50,150)
isolated = region(edges)
cv2.imshow("edges", edges)
cv2.imshow("iso", isolated)
cv2.waitKey(0)


#DRAWING LINES: (order of params) --> region of interest, bin size (P, theta), min intersections needed, placeholder array, 
lines = cv2.HoughLinesP(isolated, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
averaged_lines = average(copy, lines)
black_lines = display_lines(copy, averaged_lines)
#taking wighted sum of original image and lane lines image
lanes = cv2.addWeighted(copy, 0.8, black_lines, 1, 1)
cv2.imshow("lanes", lanes)
cv2.waitKey(0)